# Utilização de LLMs

In [1]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings(
    "ignore", category=FutureWarning, message=".*TRANSFORMERS_CACHE.*"
)
warnings.filterwarnings(
    "ignore", message=".*resume_download.*deprecated.*", category=FutureWarning
)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings(
    "ignore",
    message=".*use_reentrant parameter should be passed explicitly.*",
    category=UserWarning,
)
warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly.",
)


# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)

In [2]:
import warnings

import accelerate
import bitsandbytes
import datasets
import peft
import torch
import transformers
import trl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print("transformers version:", transformers.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(
    f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'"
)
print(
    "Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16."
)

torch version: 2.3.1
bitsandbytes version: 0.43.3
peft version: 0.12.0
accelerate version: 0.34.2
datasets version: 2.21.0
trl version: 0.10.1
transformers version: 4.46.1
Device name: 'NVIDIA GeForce RTX 4060 Ti'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Ti', major=8, minor=9, total_memory=16059MB, multi_processor_count=34)'
Suporta bfloat16.


In [3]:
import os
from random import randrange

import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import login
from peft import LoraConfig, PeftModel, TaskType, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainerCallback,
    TrainerControl,
    TrainerState,
    TrainingArguments,
    pipeline,
    set_seed,
)
from trl import SFTConfig, SFTTrainer

In [4]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages, max_new_tokens=2000, **kwargs):
        model_inputs = self.tokenize(messages)
        model_inputs["attention_mask"] = model_inputs["attention_mask"].to(
            model_inputs["input_ids"].device
        )
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            attention_mask=model_inputs["attention_mask"],
            pad_token_id=self.tokenizer.pad_token_id,
            **kwargs
        )
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
if torch.cuda.is_bf16_supported():
    # compute_dtype = torch.bfloat16
    # attn_implementation = "flash_attention_2"
    compute_dtype = torch.float16
    attn_implementation = "flash_attention_2"
    # attn_implementation = 'eager'
else:
    compute_dtype = torch.float16
    attn_implementation = "eager"

print(attn_implementation)
print(compute_dtype)

flash_attention_2
torch.float16


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token=os.environ["HUGGINGFACE_TOKEN"])

model_id = "emdemor/question-generator-v2"
commit_hash = None


# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    revision=commit_hash,
)

# # adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
# model = prepare_model_for_kbit_training(model)


def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=commit_hash)
    tokenizer.padding_side = "right"
    return tokenizer


tokenizer = set_tokenizer(model_id)

llm = LanguageModel(tokenizer, model, device="cuda")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Exemplos

In [ ]:
%%time


def generate_question(context, temperature=0.01):
    return llm.generate([{"content": f"{context}", "role": "user"}], temperature=temperature)
    

In [13]:
context = """
Esses interlocutores avaliam que transferir o vice-presidente Geraldo Alckmin do Ministério do Desenvolvimento, Indústria, Comércio e Serviços para a vaga, como cogitado, seria colocar seu vice numa posição de intermediar constantes conflitos com o governo e com o partido do presidente, o PT.
"""

result = generate_question(context, temperature=0.01)
print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))

[
    {
        "pergunta": "Onde o narrador estava indo na noite em questão?",
        "resposta": "O narrador estava indo do Engenho Novo para a cidade."
    },
    {
        "pergunta": "O que o narrador encontrou no trem da Central?",
        "resposta": "O narrador encontrou um rapaz do bairro que conhecia de vista e de chapéu."
    },
    {
        "pergunta": "Como o rapaz se apresentou ao narrador?",
        "resposta": "O rapaz cumprimentou o narrador, sentou-se ao seu lado e falou sobre a Lua e os ministros."
    },
    {
        "pergunta": "O que o rapaz fez após falar sobre a Lua e os ministros?",
        "resposta": "O rapaz acabou recitando versos para o narrador."
    },
    {
        "pergunta": "Qual era a duração da viagem e como se relacionava com a leitura dos versos?",
        "resposta": "A viagem era curta, e os versos poderiam ser que não fossem inteiramente maus."
    },
    {
        "pergunta": "O que aconteceu quando o narrador se fechou os olhos durante a 

In [14]:
result = generate_question(context, temperature=0.3)
print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))

[
    {
        "pergunta": "Onde o narrador estava indo na noite em questão?",
        "resposta": "O narrador estava indo do Engenho Novo."
    },
    {
        "pergunta": "O que o narrador encontrou no trem da Central?",
        "resposta": "O narrador encontrou um rapaz do bairro que ele conhece de vista e de chapéu."
    },
    {
        "pergunta": "Como o rapaz se apresentou ao narrador?",
        "resposta": "O rapaz cumprimentou o narrador, sentou-se ao seu lado e falou sobre a Lua e os ministros."
    },
    {
        "pergunta": "O que o rapaz fez depois de falar sobre a Lua e os ministros?",
        "resposta": "O rapaz acabou recitando versos para o narrador."
    },
    {
        "pergunta": "Qual era a duração da viagem e como se relacionava com os versos?",
        "resposta": "A viagem era curta, e os versos poderiam ser que não fossem inteiramente maus."
    },
    {
        "pergunta": "O que aconteceu quando o narrador se sentiu cansado durante a viagem?",
        

In [15]:
result = generate_question(context, temperature=0.7)
print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))

[
    {
        "pergunta": "Onde o narrador estava vindo de uma cidade?",
        "resposta": "O narrador estava vindo da cidade para o Engenho Novo."
    },
    {
        "pergunta": "O que o narrador encontrou no trem da Central?",
        "resposta": "O narrador encontrou um rapaz aqui do bairro, que ele conhece de vista e de chapéu."
    },
    {
        "pergunta": "Como o rapaz se apresentou ao narrador?",
        "resposta": "O rapaz cumprimentou-se, sentou-se ao pé de mim, falou da Lua e dos ministros, e acabou recitando versos."
    },
    {
        "pergunta": "Qual era a duração da viagem mencionada no contexto?",
        "resposta": "A viagem era curta."
    },
    {
        "pergunta": "O que aconteceu quando o narrador se fechou os olhos durante a leitura?",
        "resposta": "Quando o narrador se fechou os olhos três ou quatro vezes, o rapaz interrompeu a leitura e meteu os versos no bolso."
    },
    {
        "pergunta": "O que o narrador pensava sobre os versos qu

In [16]:
result = generate_question(context, temperature=0.9)
print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))

[
    {
        "pergunta": "O que aconteceu na noite mencionada?",
        "resposta": "Encontrei um rapaz no trem da Central, que eu conheço de vista e de chapéu."
    },
    {
        "pergunta": "O que o rapaz fez ao encontrar o narrador?",
        "resposta": "O rapaz cumprimentou o narrador, sentou-se ao seu lado e falou sobre a Lua e os ministros."
    },
    {
        "pergunta": "O que o rapaz fez ao narrador durante a viagem?",
        "resposta": "Ele começou a recitar versos ao narrador."
    },
    {
        "pergunta": "Qual foi a razão pela qual o narrador suspeitou dos versos?",
        "resposta": "O narrador suspeitou que os versos não fossem inteiramente maus, pois a viagem era curta."
    },
    {
        "pergunta": "O que aconteceu quando o narrador se fechou os olhos?",
        "resposta": "Quando o narrador se fechou os olhos três ou quatro vezes, o rapaz interrompeu a leitura e colocou os versos no bolso."
    }
]


In [18]:
result = generate_question(context, temperature=1.2)
print(json.dumps(json.loads(result), indent=4, ensure_ascii=False))

[
    {
        "pergunta": "Onde eu encontra o rapaz que cumprem apresentação no trem da Central?",
        "resposta": "No bairro do Engenho Novo."
    },
    {
        "pergunta": "Com quem eu já conheci, além do rapaz mencionado?",
        "resposta": "Pela vista e pela sua faixa."
    },
    {
        "pergunta": "O que o rapaz falava enquanto cumprimentava o autor?",
        "resposta": "- Da Lua e dos ministros."
    },
    {
        "pergunta": "O que aconteceu quando o autor se fechou os olhos no trem?",
        "resposta": "- Os versos recitados a ele foram colocados no bolso."
    },
    {
        "pergunta": "Qual foi a consequência de me fechar os olhos no trajeto do trem?",
        "resposta": "- O rapaz interrompeu a leitura e guardou seus versos."
    },
    {
        "pergunta": "A viagem do trem era de que características?",
        "resposta": "- A viagem foi curta."
    },
    {
        "pergunta": "Qual era a impressão que o autor teve sobre os versos do rapaz, con